In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install transformers bitsandbytes sentencepiece accelerate guidance --upgrade -qq

In [ ]:
!pip install --upgrade transformers -qq
!pip install accelerate
!pip install -q -U google-generativeai
!pip install --upgrade transformers -qq
!pip install accelerate
!pip install groq
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 925.5 kB/s eta 0:00:00a 0:00:01


In [ ]:
!wget -nc https://people.eecs.berkeley.edu/~hendrycks/data.tar

--2024-06-27 06:54:34--  https://people.eecs.berkeley.edu/~hendrycks/data.tar
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.244.190
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.244.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 166184960 (158M) [application/x-tar]
Saving to: 'data.tar'

data.tar            100%[===================>] 158.49M  54.0MB/s    in 2.9s    

2024-06-27 06:54:38 (54.0 MB/s) - 'data.tar' saved [166184960/166184960]



In [ ]:
# import tarfile
# unzip_path = '.'
# tar = tarfile.open('data.tar')
# tar.extractall(path=unzip_path)
# tar.close()

In [ ]:
import pandas as pd
df_reasoning = pd.read_parquet("hf://datasets/livebench/reasoning/data/test-00000-of-00001.parquet")
df_math = pd.read_parquet("hf://datasets/livebench/math/data/test-00000-of-00001.parquet")
df_coding = pd.read_parquet("hf://datasets/livebench/coding/data/test-00000-of-00001.parquet")
df_language = pd.read_parquet("hf://datasets/livebench/language/data/test-00000-of-00001.parquet")
df_data_analysis = pd.read_parquet("hf://datasets/livebench/data_analysis/data/test-00000-of-00001.parquet")
# df_instruction = pd.read_parquet("hf://datasets/livebench/instruction_following/data/test-00000-of-00001.parquet")

In [ ]:
df_reasoning.to_csv('reasoning_dev.csv', index=False)
df_math.to_csv('math_dev.csv', index=False)
df_language.to_csv('language_dev.csv', index=False)
df_data_analysis.to_csv('data_analysis_dev.csv', index=False)
df_coding.to_csv('coding_dev.csv', index=False)

In [ ]:
print(df_reasoning.shape)
print(df_math.shape)
print(df_coding.shape)
print(df_language.shape)
print(df_data_analysis.shape)

(100, 5)
(232, 9)
(88, 13)
(140, 10)
(150, 5)


In [ ]:
import os

dev_folder_path = '/kaggle/working/dev'
os.makedirs(dev_folder_path, exist_ok=True)

In [ ]:
import os
import shutil

source_folder = '/kaggle/working/'
destination_folder = '/kaggle/working/dev'

files = os.listdir(source_folder)
csv_files = [file for file in files if file.endswith('.csv')]

for file in csv_files:
    source_file_path = os.path.join(source_folder, file)
    destination_file_path = os.path.join(destination_folder, file)
    shutil.move(source_file_path, destination_file_path)
    print(f"Moved '{file}' to '{destination_folder}'")

Moved 'data_analysis_dev.csv' to '/kaggle/working/dev'
Moved 'language_dev.csv' to '/kaggle/working/dev'
Moved 'math_dev.csv' to '/kaggle/working/dev'
Moved 'reasoning_dev.csv' to '/kaggle/working/dev'
Moved 'coding_dev.csv' to '/kaggle/working/dev'


In [ ]:
import torch
import numpy as np
from sklearn.isotonic import IsotonicRegression
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification, pipeline
from transformers import BitsAndBytesConfig
import torch.nn.functional as F
from transformers import pipeline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
import google.generativeai as genai
from bert_score import score
import os
from groq import Groq
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import json
import re
import concurrent.futures
from tqdm import tqdm
from yaml import safe_load

2024-06-27 06:55:03.053453: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-27 06:55:03.053590: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-27 06:55:03.312739: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
import google.generativeai as genai
import os
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


os.environ['GOOGLE_API_KEY']=""
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(filtered_tokens)

In [ ]:
client = Groq(api_key="")

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What is the meaning of life? How modern Ai is affeting the real life and what will be the Ai outcomes in future?",
        }

    ],
    model="llama3-70b-8192",
)

print(chat_completion.choices[0].message.content)

What a profound and existential question!

**What is the meaning of life?**

The meaning of life is a philosophical question that has been debated and explored by scholars, thinkers, and individuals across various cultures and centuries.asements.<|start_header_id|><|start_header_id|>assistant<|end_header_id|>

While there's no one definitive answer, here are some insights:

1. **Purpose and Fulfillment**: Many believe that the meaning of life is to find one's purpose and fulfill it.<|start_header_id|><|start_header_id|>assistant<|end_header_id|>
2. **Happiness and Well-being**: Others argue that the meaning of life is to attain happiness and well-being, often through personal growth, relationships, and contributing to the greater good.
3. **Self-Discovery and Exploration**: Some propose that the meaning of life is to continuously learn, grow, and explore one's own potential and the world around them


In [ ]:
gemini_model = genai.GenerativeModel('gemini-pro')

## Zero shot prompting with livebench and mmlu

In [ ]:
class Tokenizer:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def encode(self, text, max_length=1024):
        return self.tokenizer.encode_plus(text, return_tensors='pt', max_length=max_length, truncation=True)

    def decode(self, tokens):
        return self.tokenizer.decode(tokens, skip_special_tokens=True)

In [ ]:
class EvaluationPipeline:
    def __init__(self, api_key, judge_model_name, smaller_model_name, temperature=0.8, max_tokens=300):
        self.api_key = api_key
        self.judge_model_name = judge_model_name
        self.smaller_model_name = smaller_model_name
        self.temperature = temperature
        self.max_tokens = max_tokens

        self.tokenizer = AutoTokenizer.from_pretrained(smaller_model_name)

        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )

        self.main_model = AutoModelForCausalLM.from_pretrained(
            smaller_model_name,
            torch_dtype=torch.float16,
            quantization_config=bnb_config,
            low_cpu_mem_usage=True,
            device_map="auto",
        )

        self.mmlu_categories = {
            "abstract_algebra": ["math"],
            "anatomy": ["health"],
            "astronomy": ["physics"],
            "business_ethics": ["business"],
            "clinical_knowledge": ["health"],
            "college_biology": ["biology"],
            "college_chemistry": ["chemistry"],
            "college_computer_science": ["computer science"],
            "college_mathematics": ["math"],
            "college_medicine": ["health"],
            "college_physics": ["physics"],
            "computer_security": ["computer science"],
            "conceptual_physics": ["physics"],
            "econometrics": ["economics"],
            "electrical_engineering": ["engineering"],
            "elementary_mathematics": ["math"],
            "formal_logic": ["philosophy"],
            "global_facts": ["other"],
            "high_school_biology": ["biology"],
            "high_school_chemistry": ["chemistry"],
            "high_school_computer_science": ["computer science"],
            "high_school_european_history": ["history"],
            "high_school_geography": ["geography"],
            "high_school_government_and_politics": ["politics"],
            "high_school_macroeconomics": ["economics"],
            "high_school_mathematics": ["math"],
            "high_school_microeconomics": ["economics"],
            "high_school_physics": ["physics"],
            "high_school_psychology": ["psychology"],
            "high_school_statistics": ["math"],
            "high_school_us_history": ["history"],
            "high_school_world_history": ["history"],
            "human_aging": ["health"],
            "human_sexuality": ["culture"],
            "international_law": ["law"],
            "jurisprudence": ["law"],
            "logical_fallacies": ["philosophy"],
            "machine_learning": ["computer science"],
            "management": ["business"],
            "marketing": ["business"],
            "medical_genetics": ["health"],
            "miscellaneous": ["other"],
            "moral_disputes": ["philosophy"],
            "moral_scenarios": ["philosophy"],
            "nutrition": ["health"],
            "philosophy": ["philosophy"],
            "prehistory": ["history"],
            "professional_accounting": ["other"],
            "professional_law": ["law"],
            "professional_medicine": ["health"],
            "professional_psychology": ["psychology"],
            "public_relations": ["politics"],
            "security_studies": ["politics"],
            "sociology": ["culture"],
            "us_foreign_policy": ["politics"],
            "virology": ["health"],
            "world_religions": ["philosophy"],
        }

        self.livebench_categories = {
            "data_analysis": ["data_analysis"],
            "coding": ["coding"],
            "language": ["language"],
            "reasoning": ["reasoning"],
            "math": ["math"],
        }

    def get_model_answer(self, prompt):
        inputs = self.tokenizer.encode(prompt, return_tensors="pt").to(self.main_model.device)
        outputs = self.main_model.generate(
            inputs,
            max_new_tokens=300,
            num_return_sequences=1,
            temperature=self.temperature,
            output_scores=True,
            return_dict_in_generate=True
        )
        response = self.tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

        logits = torch.stack(outputs.scores, dim=1)
        probs = F.softmax(logits, dim=-1)

        token_ids = outputs.sequences[:, inputs.shape[1]:]
        confidences = probs.gather(2, token_ids.unsqueeze(-1)).squeeze(-1).mean(dim=1).detach().cpu().numpy()

        avg_confidence = confidences[0]
        return response, avg_confidence

    def judge_answer(self, response, prompt):
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"""
                        Review the user’s question and the corresponding response using the binary scoring system described below.
                        - 0 points: The response is incorrect or does not address the user’s question.
                        - 1 point: The response is correct and addresses the user’s question.

                        User: {prompt}
                        Response: {response}
                        """
                }
            ],
            model=self.judge_model_name,
        )

        judge_response = chat_completion.choices[0].message.content.strip()
        return judge_response

    def parse_evaluation(self, evaluation):
        return 1 if "1 point" in evaluation else 0

    def generate_reference_text(self, prompt):
        inputs = self.tokenizer.encode(prompt, return_tensors="pt").to(self.main_model.device)
        outputs = self.main_model.generate(
            inputs,
            max_new_tokens=self.max_tokens,
            num_return_sequences=1,
            temperature=self.temperature,
            do_sample=True
        )
        reference_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return reference_text


    def calculate_bertscore(self, references, candidates):
        references = list(references)
        candidates = list(candidates)
        P, R, F1 = score(candidates, references, lang="en", model_type="bert-base-uncased")
        return P.mean().item(), R.mean().item(), F1.mean().item()

    def evaluate_from_csv(self, csv_file_path, dataset_type, num_questions=None):
        self.prompts = []
        self.responses = []
        self.references = []
        self.confidences = []
        self.accuracies = []

        df = pd.read_csv(csv_file_path)

        if dataset_type == "mmlu":
            self.prompts = df.iloc[:, 0].tolist()
            if num_questions:
                self.prompts = self.prompts[:num_questions]

            for prompt in self.prompts:
                response, confidence = self.get_model_answer(prompt)
                self.responses.append(response)
                self.confidences.append(confidence)
                reference_text = self.generate_reference_text(prompt)
                self.references.append(reference_text)
                judgement = self.judge_answer(response, prompt)
                accuracy = self.parse_evaluation(judgement)
                self.accuracies.append(accuracy)

        elif dataset_type == "livebench":
            if "coding" in csv_file_path:
                self.prompts = df["turns"].tolist()
                self.references = df["solution"].tolist()
            else:
                self.prompts = df["turns"].tolist()
                self.references = df["ground_truth"].tolist()

            # Filter out rows where "turns" has a value but "ground_truth"/"solution" is empty
            self.prompts, self.references = zip(*[
                (prompt, reference) for prompt, reference in zip(self.prompts, self.references)
                if pd.notna(prompt) and pd.notna(reference)
            ])

            if num_questions:
                self.prompts = self.prompts[:num_questions]
                self.references = self.references[:num_questions]

            for prompt, reference in zip(self.prompts, self.references):
                response, confidence = self.get_model_answer(prompt)
                self.responses.append(response)
                self.confidences.append(confidence)
                accuracy = 1 if response.strip() == reference.strip() else 0
                self.accuracies.append(accuracy)

        precision, recall, f1 = self.calculate_bertscore(self.references, self.responses)
        print(f"BERTScore - Precision: {precision}, Recall: {recall}, F1: {f1}")

        data, bins, bin_accuracies = self.calculate_ece()
        data.to_csv("results.csv", index=False)

        # Save BERTScore along with ECE results
        data['precision'] = precision
        data['recall'] = recall
        data['f1'] = f1
        data.to_csv("results.csv", index=False)

        return data, bins, bin_accuracies

    def evaluate_folder(self, folder_path, dataset_type, num_questions=None):
        results = {}
        reliability_data = {}
        category_results = {category: [] for category in set(cat for sublist in self.mmlu_categories.values() for cat in sublist)}

        if dataset_type == "livebench":
            category_results = {category: [] for category in set(cat for sublist in self.livebench_categories.values() for cat in sublist)}

        for filename in os.listdir(folder_path):
            if filename.endswith(".csv"):
                filepath = os.path.join(folder_path, filename)
#                 class_name = filename.split('.')[0]
                class_name = filename[:-8]
                data, bins, bin_accuracies = self.evaluate_from_csv(filepath, dataset_type, num_questions)
                results[class_name] = data


                # Store class-wise data for reliability diagram
                reliability_data[class_name] = {
                    'bin_confidences': (bins[:-1] + bins[1:]) / 2,
                    'bin_accuracies': bin_accuracies
                }
                if dataset_type == "mmlu":
                    for category_name in self.mmlu_categories.get(class_name, []):
                        # Calculate ECE within the category loop
                        bin_confidences = (bins[:-1] + bins[1:]) / 2  # Confidences for each bin
                        valid_bins = bin_accuracies.dropna().index  # Bins with data
                        bin_accuracies = bin_accuracies[valid_bins]
                        bin_proportions = data['bin'].value_counts(normalize=True)[valid_bins]  # Get proportions for valid bins
                        bin_confidences = bin_confidences[valid_bins]  # Select confidences for valid bins

                        ece = np.sum(np.abs(bin_accuracies - bin_confidences) * bin_proportions)
                        category_results[category_name].append(ece)
                        print(f"{class_name} ECE: {ece}")

                elif dataset_type == "livebench":
                    for category_name in self.livebench_categories.get(class_name, []):
                        # Calculate ECE within the category loop
                        bin_confidences = (bins[:-1] + bins[1:]) / 2  # Confidences for each bin
                        valid_bins = bin_accuracies.dropna().index  # Bins with data
                        bin_accuracies = bin_accuracies[valid_bins]
                        bin_proportions = data['bin'].value_counts(normalize=True)[valid_bins]  # Get proportions for valid bins
                        bin_confidences = bin_confidences[valid_bins]  # Select confidences for valid bins

                        ece = np.sum(np.abs(bin_accuracies - bin_confidences) * bin_proportions)
                        category_results[category_name].append(ece)
                        print(f"{class_name} ECE: {ece}")


                print(f"Processed {class_name}")


        # Calculate average ECE for each category
        average_ece_results = {category: np.mean(ece_list) for category, ece_list in category_results.items()}
        # Save average ECE results to 'category_results.csv'
        average_ece_df = pd.DataFrame({'Category': average_ece_results.keys(), 'Average ECE': average_ece_results.values()})
        average_ece_df.to_csv("category_results.csv", index=False)


        # Save BERTScore results for all classes
        bertscore_results = {
            class_name: {
                'precision': results[class_name]['precision'].iloc[0],
                'recall': results[class_name]['recall'].iloc[0],
                'f1': results[class_name]['f1'].iloc[0]
            }
            for class_name in results
        }
        bertscore_df = pd.DataFrame.from_dict(bertscore_results, orient='index')
        bertscore_df.to_csv("bertscore_results.csv", index=True)

        # Plot reliability diagram for all classes in one figure
        plt.figure(figsize=(10, 6))
        plt.plot([0, 1], [0, 1], 'k--', label='Perfectly Calibrated')

        # Find the maximum number of bins across all classes
        max_bins = max([len(data['bin_accuracies']) for data in reliability_data.values()])

        # Pad bin_accuracies with NaN to ensure equal lengths for plotting
        for class_name, data in reliability_data.items():
            num_missing_bins = max_bins - len(data['bin_accuracies'])
            data['bin_accuracies'] = np.pad(data['bin_accuracies'], (0, num_missing_bins), 'constant', constant_values=np.nan)

            plt.scatter(data['bin_confidences'][:max_bins], data['bin_accuracies'], label=class_name, s=50)

        plt.xlabel('Confidence')
        plt.ylabel('Accuracy')
        plt.title('Reliability Diagram for All Classes')
        plt.legend()
        plt.grid(True)
        plt.show()

    def calculate_ece(self):
        if len(self.prompts) == len(self.responses) == len(self.confidences) == len(self.accuracies):
            data = pd.DataFrame({
                'prompt': self.prompts,
                'response': self.responses,
                'confidence': self.confidences,
                'rating': self.accuracies
            })
        else:
            raise ValueError("All arrays must be of the same length")

        data['confidence_normalized'] = data['confidence'] / data['confidence'].max()

        bins = np.linspace(0, 1, 11)
        data['bin'] = pd.cut(data['confidence_normalized'], bins=bins, labels=False, include_lowest=True)

        bin_accuracies = data.groupby('bin')['rating'].mean()
        bin_proportions = data['bin'].value_counts(normalize=True)

        valid_bins = bin_accuracies.dropna().index
        bin_accuracies = bin_accuracies[valid_bins]
        bin_proportions = bin_proportions[valid_bins]
        bin_confidences = (bins[:-1] + bins[1:]) / 2
        bin_confidences = bin_confidences[valid_bins]

        bin_confidences = bin_confidences[:len(bin_accuracies)]

        ece = np.sum(np.abs(bin_accuracies - bin_confidences) * bin_proportions)
        print(f"Expected Calibration Error (ECE): {ece}")

        return data, bins, bin_accuracies

In [ ]:
## All Models
models = [
    {'name': 'Qwen/Qwen2-7B-Instruct'},
    {'name': 'stabilityai/StableBeluga-7B'},
    {'name': 'meta-llama/Meta-Llama-3-8B'},
    {'name': 'teknium/OpenHermes-2.5-Mistral-7B'},
    {'name': 'mistralai/Mistral-7B-Instruct-v0.2'},
    {'name': 'HuggingFaceH4/zephyr-7b-beta'},
]

In [1]:
# Example usage
pipeline = EvaluationPipeline(api_key="Your_api_key", judge_model_name="llama3-70b-8192", smaller_model_name=models[4]['name'])
# pipeline.evaluate_folder("/kaggle/working/dev", dataset_type="livebench", num_questions=3) # if u control question
pipeline.evaluate_folder("/kaggle/working/dev", dataset_type="livebench")

## Chain of thoughts

In [ ]:
class Tokenizer:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def encode(self, text, max_length=1024):
        return self.tokenizer.encode_plus(text, return_tensors='pt', max_length=max_length, truncation=True)

    def decode(self, tokens):
        return self.tokenizer.decode(tokens, skip_special_tokens=True)

In [ ]:
class EvaluationPipeline:
    def __init__(self, api_key, judge_model_name, smaller_model_name, temperature=0.8, max_tokens=300):
        self.api_key = api_key
        self.judge_model_name = judge_model_name
        self.smaller_model_name = smaller_model_name
        self.temperature = temperature
        self.max_tokens = max_tokens

        self.tokenizer = AutoTokenizer.from_pretrained(smaller_model_name)

        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )

        self.main_model = AutoModelForCausalLM.from_pretrained(
            smaller_model_name,
            torch_dtype=torch.float16,
            quantization_config=bnb_config,
            low_cpu_mem_usage=True,
            device_map="auto",
        )

        self.mmlu_categories = {
            "abstract_algebra": ["math"],
            "anatomy": ["health"],
            "astronomy": ["physics"],
            "business_ethics": ["business"],
            "clinical_knowledge": ["health"],
            "college_biology": ["biology"],
            "college_chemistry": ["chemistry"],
            "college_computer_science": ["computer science"],
            "college_mathematics": ["math"],
            "college_medicine": ["health"],
            "college_physics": ["physics"],
            "computer_security": ["computer science"],
            "conceptual_physics": ["physics"],
            "econometrics": ["economics"],
            "electrical_engineering": ["engineering"],
            "elementary_mathematics": ["math"],
            "formal_logic": ["philosophy"],
            "global_facts": ["other"],
            "high_school_biology": ["biology"],
            "high_school_chemistry": ["chemistry"],
            "high_school_computer_science": ["computer science"],
            "high_school_european_history": ["history"],
            "high_school_geography": ["geography"],
            "high_school_government_and_politics": ["politics"],
            "high_school_macroeconomics": ["economics"],
            "high_school_mathematics": ["math"],
            "high_school_microeconomics": ["economics"],
            "high_school_physics": ["physics"],
            "high_school_psychology": ["psychology"],
            "high_school_statistics": ["math"],
            "high_school_us_history": ["history"],
            "high_school_world_history": ["history"],
            "human_aging": ["health"],
            "human_sexuality": ["culture"],
            "international_law": ["law"],
            "jurisprudence": ["law"],
            "logical_fallacies": ["philosophy"],
            "machine_learning": ["computer science"],
            "management": ["business"],
            "marketing": ["business"],
            "medical_genetics": ["health"],
            "miscellaneous": ["other"],
            "moral_disputes": ["philosophy"],
            "moral_scenarios": ["philosophy"],
            "nutrition": ["health"],
            "philosophy": ["philosophy"],
            "prehistory": ["history"],
            "professional_accounting": ["other"],
            "professional_law": ["law"],
            "professional_medicine": ["health"],
            "professional_psychology": ["psychology"],
            "public_relations": ["politics"],
            "security_studies": ["politics"],
            "sociology": ["culture"],
            "us_foreign_policy": ["politics"],
            "virology": ["health"],
            "world_religions": ["philosophy"],
        }

        self.livebench_categories = {
            "data_analysis": ["data_analysis"],
            "coding": ["coding"],
            "language": ["language"],
            "reasoning": ["reasoning"],
            "math": ["math"],
        }

    def generate_cot_prompt(self, question):
        # Chain of Thought prompt
        cot_prompt = f"Q: {question}\nA: Let's think through this step by step.\n"
        cot_prompt += "First, consider the main aspects of the question. "
        cot_prompt += "Next, break down the problem into smaller parts. "
        cot_prompt += "Finally, synthesize the information to form a coherent answer."
        return cot_prompt

    def get_model_answer(self, question):
        """Generates an answer from the specified model using Chain of Thought prompting."""
        prompt = self.generate_cot_prompt(question)
        inputs = self.tokenizer(prompt, return_tensors='pt').to(self.main_model.device)
        outputs = self.main_model.generate(
            inputs['input_ids'], 
            max_new_tokens=300,  # Limit the number of generated tokens
            num_return_sequences=1, 
            temperature=self.temperature, 
            output_scores=True, 
            return_dict_in_generate=True
        )
        response = self.tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

        # Get the confidence score
        logits = torch.stack(outputs.scores, dim=1)
        probs = F.softmax(logits, dim=-1)

        token_ids = outputs.sequences[:, inputs['input_ids'].shape[1]:]
        confidences = probs.gather(2, token_ids.unsqueeze(-1)).squeeze(-1).mean(dim=1).detach().cpu().numpy()

        avg_confidence = confidences[0]
        return response, avg_confidence

    def judge_answer(self, response, prompt):
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"""
                        Review the user’s question and the corresponding response using the binary scoring system described below.
                        - 0 points: The response is incorrect or does not address the user’s question.
                        - 1 point: The response is correct and addresses the user’s question.

                        User: {prompt}
                        Response: {response}
                        """
                }
            ],
            model=self.judge_model_name,
        )

        judge_response = chat_completion.choices[0].message.content.strip()
        return judge_response

    def parse_evaluation(self, evaluation):
        return 1 if "1 point" in evaluation else 0

    def generate_reference_text(self, prompt):
        inputs = self.tokenizer.encode(prompt, return_tensors="pt").to(self.main_model.device)
        outputs = self.main_model.generate(
            inputs,
            max_new_tokens=self.max_tokens,
            num_return_sequences=1,
            temperature=self.temperature,
            do_sample=True
        )
        reference_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return reference_text


    def calculate_bertscore(self, references, candidates):
        references = list(references)
        candidates = list(candidates)
        P, R, F1 = score(candidates, references, lang="en", model_type="bert-base-uncased")
        return P.mean().item(), R.mean().item(), F1.mean().item()

    def evaluate_from_csv(self, csv_file_path, dataset_type, num_questions=None):
        self.prompts = []
        self.responses = []
        self.references = []
        self.confidences = []
        self.accuracies = []

        df = pd.read_csv(csv_file_path)

        if dataset_type == "mmlu":
            self.prompts = df.iloc[:, 0].tolist()
            if num_questions:
                self.prompts = self.prompts[:num_questions]
                
            for prompt in self.prompts:
                response, confidence = self.get_model_answer(prompt)
                self.responses.append(response)
                self.confidences.append(confidence)
                reference_text = self.generate_reference_text(prompt)
                self.references.append(reference_text)
                judgement = self.judge_answer(response, prompt)
                accuracy = self.parse_evaluation(judgement)
                self.accuracies.append(accuracy)
                
        elif dataset_type == "livebench":
            if "coding" in csv_file_path:
                self.prompts = df["turns"].tolist()
                self.references = df["solution"].tolist()
            else:
                self.prompts = df["turns"].tolist()
                self.references = df["ground_truth"].tolist()
            
            # Filter out rows where "turns" has a value but "ground_truth"/"solution" is empty
            self.prompts, self.references = zip(*[
                (prompt, reference) for prompt, reference in zip(self.prompts, self.references) 
                if pd.notna(prompt) and pd.notna(reference)
            ])

            if num_questions:
                self.prompts = self.prompts[:num_questions]
                self.references = self.references[:num_questions]

            for prompt, reference in zip(self.prompts, self.references):
                response, confidence = self.get_model_answer(prompt)
                self.responses.append(response)
                self.confidences.append(confidence)
                accuracy = 1 if response.strip() == reference.strip() else 0
                self.accuracies.append(accuracy)

        precision, recall, f1 = self.calculate_bertscore(self.references, self.responses)
        print(f"BERTScore - Precision: {precision}, Recall: {recall}, F1: {f1}")

        data, bins, bin_accuracies = self.calculate_ece()
        data.to_csv("results.csv", index=False)
        
        # Save BERTScore along with ECE results
        data['precision'] = precision
        data['recall'] = recall
        data['f1'] = f1
        data.to_csv("results.csv", index=False)
        
        return data, bins, bin_accuracies

    def evaluate_folder(self, folder_path, dataset_type, num_questions=None):
        results = {}
        reliability_data = {}
        category_results = {category: [] for category in set(cat for sublist in self.mmlu_categories.values() for cat in sublist)}

        if dataset_type == "livebench":
            category_results = {category: [] for category in set(cat for sublist in self.livebench_categories.values() for cat in sublist)}

        for filename in os.listdir(folder_path):
            if filename.endswith(".csv"):
                filepath = os.path.join(folder_path, filename)
#                 class_name = filename.split('.')[0]
                class_name = filename[:-8] 
                data, bins, bin_accuracies = self.evaluate_from_csv(filepath, dataset_type, num_questions)
                results[class_name] = data
        
                
                # Store class-wise data for reliability diagram
                reliability_data[class_name] = {
                    'bin_confidences': (bins[:-1] + bins[1:]) / 2,
                    'bin_accuracies': bin_accuracies
                }
                
                if dataset_type == "mmlu":
                    for category_name in self.mmlu_categories.get(class_name, []):
                        print(f"Processed {class_name}")
                        # Calculate ECE within the category loop
                        bin_confidences = (bins[:-1] + bins[1:]) / 2  # Confidences for each bin
                        valid_bins = bin_accuracies.dropna().index  # Bins with data
                        bin_accuracies = bin_accuracies[valid_bins]
                        bin_proportions = data['bin'].value_counts(normalize=True)[valid_bins]  # Get proportions for valid bins
                        bin_confidences = bin_confidences[valid_bins]  # Select confidences for valid bins

                        ece = np.sum(np.abs(bin_accuracies - bin_confidences) * bin_proportions)
                        category_results[category_name].append(ece)
                        print(f"{class_name} ECE: {ece}")

                elif dataset_type == "livebench":
                    for category_name in self.livebench_categories.get(class_name, []):
                        print(f"Processed {class_name}")
                        # Calculate ECE within the category loop
                        bin_confidences = (bins[:-1] + bins[1:]) / 2  # Confidences for each bin
                        valid_bins = bin_accuracies.dropna().index  # Bins with data
                        bin_accuracies = bin_accuracies[valid_bins]
                        bin_proportions = data['bin'].value_counts(normalize=True)[valid_bins]  # Get proportions for valid bins
                        bin_confidences = bin_confidences[valid_bins]  # Select confidences for valid bins

                        ece = np.sum(np.abs(bin_accuracies - bin_confidences) * bin_proportions)
                        category_results[category_name].append(ece)
                        print(f"{class_name} ECE: {ece}")
                        

        # Calculate average ECE for each category
        average_ece_results = {category: np.mean(ece_list) for category, ece_list in category_results.items()}

        # Save average ECE results to 'category_results.csv'
        average_ece_df = pd.DataFrame({'Category': average_ece_results.keys(), 'Average ECE': average_ece_results.values()})
        average_ece_df.to_csv("category_results.csv", index=False)

        
        # Save BERTScore results for all classes
        bertscore_results = {
            class_name: {
                'precision': results[class_name]['precision'].iloc[0],
                'recall': results[class_name]['recall'].iloc[0],
                'f1': results[class_name]['f1'].iloc[0]
            }
            for class_name in results
        }
        bertscore_df = pd.DataFrame.from_dict(bertscore_results, orient='index')
        bertscore_df.to_csv("bertscore_results.csv", index=True)
        
        # Plot reliability diagram for all classes in one figure
        plt.figure(figsize=(10, 6))
        plt.plot([0, 1], [0, 1], 'k--', label='Perfectly Calibrated')

        # Find the maximum number of bins across all classes
        max_bins = max([len(data['bin_accuracies']) for data in reliability_data.values()])

        # Pad bin_accuracies with NaN to ensure equal lengths for plotting
        for class_name, data in reliability_data.items():
            num_missing_bins = max_bins - len(data['bin_accuracies'])
            data['bin_accuracies'] = np.pad(data['bin_accuracies'], (0, num_missing_bins), 'constant', constant_values=np.nan)

            plt.scatter(data['bin_confidences'][:max_bins], data['bin_accuracies'], label=class_name, s=50)

        plt.xlabel('Confidence')
        plt.ylabel('Accuracy')
        plt.title('Reliability Diagram for All Classes')
        plt.legend()
        plt.grid(True)
        plt.show()

    def calculate_ece(self):
        if len(self.prompts) == len(self.responses) == len(self.confidences) == len(self.accuracies):
            data = pd.DataFrame({
                'prompt': self.prompts,
                'response': self.responses,
                'confidence': self.confidences,
                'rating': self.accuracies
            })
        else:
            raise ValueError("All arrays must be of the same length")

        data['confidence_normalized'] = data['confidence'] / data['confidence'].max()

        bins = np.linspace(0, 1, 11)
        data['bin'] = pd.cut(data['confidence_normalized'], bins=bins, labels=False, include_lowest=True)

        bin_accuracies = data.groupby('bin')['rating'].mean()
        bin_proportions = data['bin'].value_counts(normalize=True)

        valid_bins = bin_accuracies.dropna().index
        bin_accuracies = bin_accuracies[valid_bins]
        bin_proportions = bin_proportions[valid_bins]
        bin_confidences = (bins[:-1] + bins[1:]) / 2
        bin_confidences = bin_confidences[valid_bins]

        bin_confidences = bin_confidences[:len(bin_accuracies)]

        ece = np.sum(np.abs(bin_accuracies - bin_confidences) * bin_proportions)
        print(f"Expected Calibration Error (ECE): {ece}")

        return data, bins, bin_accuracies

In [ ]:
## All Models
models = [
    {'name': 'Qwen/Qwen2-7B-Instruct'},
    {'name': 'stabilityai/StableBeluga-7B'},
    {'name': 'meta-llama/Meta-Llama-3-8B'}, 
    {'name': 'teknium/OpenHermes-2.5-Mistral-7B'}, 
    {'name': 'mistralai/Mistral-7B-Instruct-v0.2'},  
    {'name': 'HuggingFaceH4/zephyr-7b-beta'},
]

# Example usage
pipeline = EvaluationPipeline(api_key="Your_api_key", judge_model_name="llama3-70b-8192", smaller_model_name=models[4]['name'])
# pipeline.evaluate_folder("/kaggle/working/dev", dataset_type="livebench", num_questions=1) # if u control question
pipeline.evaluate_folder("/kaggle/working/dev", dataset_type="livebench")